In [57]:
import pandas as pd
import os, re
from helpers import *
import datetime

In [58]:
############
# input VARS

PERSON = "taylor"
TRYS = 1
VALID_LABELS = ["lights-on", "turn-off"]
NUM_CHANNELS = 22
TODAY = False

############
# const VARS

DATE = datetime.datetime.today().strftime('%m_%d')
if TODAY == True:
    FNAME = "{}_{}_{}".format(PERSON,DATE,TRYS)
else:
    FNAME = "osc_test2"
ROOT = os.getcwd() + "/images_scaled/" + FNAME
OUTPUT = os.getcwd() + "/train_csv/" + FNAME + ".csv"
DATE_PATTERN = "[0-9]{2}_[0-9]{2}"
IMG_EXT = ".png"
VERBOSITY = 100
CATS, MONTHS, DAYS, LABELS, SEQ, SETS = [], [], [], [], [], []
for i in range(1, NUM_CHANNELS+1):
    globals()["PATH{}".format(i)] = []
    
print(ROOT)

/Users/kyy/cerebro_train/images_scaled/osc_test2


In [59]:
def make_df_from_images(image_root):
    flag = 1
    for cat in [d for d in os.listdir(image_root) if "voice" in d]:
        cat_path = os.path.join(image_root, cat)
        for date in [d for d in os.listdir(cat_path) if re.match(DATE_PATTERN, d)]:
            print("\tProcessing {}".format(date))
            date_path = os.path.join(cat_path, date)
            month = int(date[:2])
            day = int(date[3:])
            date_count = 0
            for label in [d for d in os.listdir(date_path) if d in VALID_LABELS]:
                label_path = os.path.join(date_path, label)
                placeholder = os.path.join(label_path, "ch1")
                for image in [f for f in os.listdir(placeholder) if f.endswith(IMG_EXT)]:
                    date_count += 1
                    for i in range(1, NUM_CHANNELS+1):
                        p = os.path.join(label_path, "ch{}".format(i), image)
                        if os.path.exists(p):
                            globals()["PATH{}".format(i)].append(p)
                        else:
                            globals()["PATH{}".format(i)].append(float('nan'))
                    CATS.append(cat)
                    DAYS.append(day)
                    MONTHS.append(month)
                    LABELS.append(label)
                    sequence_number = int(image[:-4])
                    basenum = sequence_number % 10
                    SEQ.append(sequence_number)
                    if basenum < 4:
                        SETS.append("Training")
                    elif basenum < 5:
                        if flag == 1:
                            SETS.append("Validation")
                            flag = 0
                        else:
                            SETS.append('Testing')
                            flag = 1
#                     else:
#                         SETS.append("Testing")
            print("\t\tProcessed {} sequences".format(date_count))
    d = {
            "Category":CATS,
            "Day":DAYS,
            "Month":MONTHS,
            "Label":LABELS,
            "SequenceNumber":SEQ,
            "Set":SETS
        }
    for i in range(1, NUM_CHANNELS+1):
        d["Path{}".format(i)] = globals()["PATH{}".format(i)]
    return pd.DataFrame(d)

In [60]:
####################################
# turn spectrogram paths to csv file

df = timer(make_df_from_images, ROOT)
df.to_csv(OUTPUT, index=False)

Start: 2018-12-12 05:57:16.830901
	Processing 12_11
		Processed 4400 sequences
	Processing 12_12
		Processed 4395 sequences
End: 2018-12-12 05:57:19.033661
Finished in 00:00:00:02.203


In [61]:
print(set(df['Set']))
from collections import Counter
c = Counter(df['Set'])

print( c.items() )

{'Testing', 'Validation', 'Training'}
dict_items([('Training', 7036), ('Validation', 880), ('Testing', 879)])


In [62]:
print(df.tail(5))

      Category  Day  Month     Label  SequenceNumber         Set  \
8790  no_voice   12     12  turn-off            1094  Validation   
8791  no_voice   12     12  turn-off            3683    Training   
8792  no_voice   12     12  turn-off            1080    Training   
8793  no_voice   12     12  turn-off            3873    Training   
8794  no_voice   12     12  turn-off            1902    Training   

                                                  Path1  \
8790  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8791  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8792  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8793  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8794  /Users/kyy/cerebro_train/images_scaled/osc_tes...   

                                                  Path2  \
8790  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8791  /Users/kyy/cerebro_train/images_scaled/osc_tes...   
8792  /Users/kyy/cerebro_train/images_scaled/osc_tes...   


In [63]:
print(df.describe())

               Day   Month  SequenceNumber  Path5  Path6  Path7  Path8  Path9  \
count  8795.000000  8795.0     8795.000000    0.0    0.0    0.0    0.0    0.0   
mean     11.499716    12.0     2195.752132    NaN    NaN    NaN    NaN    NaN   
std       0.500028     0.0     1269.522275    NaN    NaN    NaN    NaN    NaN   
min      11.000000    12.0        0.000000    NaN    NaN    NaN    NaN    NaN   
25%      11.000000    12.0     1094.000000    NaN    NaN    NaN    NaN    NaN   
50%      11.000000    12.0     2194.000000    NaN    NaN    NaN    NaN    NaN   
75%      12.000000    12.0     3293.500000    NaN    NaN    NaN    NaN    NaN   
max      12.000000    12.0     4394.000000    NaN    NaN    NaN    NaN    NaN   

       Path10  Path11   ...    Path13  Path14  Path15  Path16  Path17  Path18  \
count     0.0     0.0   ...       0.0     0.0     0.0     0.0     0.0     0.0   
mean      NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   
std       NaN     NaN   ...

In [64]:
df

,Category,Day,Month,Label,SequenceNumber,Set,Path1,Path2,Path3,Path4,...,Path13,Path14,Path15,Path16,Path17,Path18,Path19,Path20,Path21,Path22
0,no_voice,11,12,lights-on,2590,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no_voice,11,12,lights-on,3842,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no_voice,11,12,lights-on,2584,Validation,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no_voice,11,12,lights-on,393,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no_voice,11,12,lights-on,1933,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,no_voice,11,12,lights-on,1700,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,no_voice,11,12,lights-on,3671,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,no_voice,11,12,lights-on,1072,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,no_voice,11,12,lights-on,1714,Testing,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,no_voice,11,12,lights-on,3103,Training,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,/Users/kyy/cerebro_train/images_scaled/osc_tes...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
